In [1]:
# Enter your CERN password to get access to the data files.
# You only need to run this once per SWAN session.

import getpass
import os
username = os.environ['USER']
if os.system('klist | grep Default | grep ' + username + '@CERN.CH'):
    os.system('echo %s' % getpass.getpass() + " | kinit " + username)

In [2]:
import ROOT

Welcome to JupyROOT 6.24/00


In [3]:
file = ROOT.TFile.Open("root://eosproject//eos/project/h/highlo/workshop/ZS_ZC/messages/20150205.root")

In [4]:
# Look, there's a TTree inside!
file.ls()

TNetXNGFile**		root://eosproject//eos/project/h/highlo/workshop/ZS_ZC/messages/20150205.root	
 TNetXNGFile*		root://eosproject//eos/project/h/highlo/workshop/ZS_ZC/messages/20150205.root	
  KEY: TTree	LOB;11	LOB [current cycle]
  KEY: TTree	LOB;10	LOB [backup cycle]


In [5]:
# Create an RDataFrame object on the TTree called "LOB"
ROOT.EnableImplicitMT() # with concurrency
rdf = ROOT.RDataFrame("LOB", file)

Warning in <RTaskArenaWrapper>: CPU Bandwith Control Active. Proceeding with 2 threads accordingly


In [6]:
# Create a canvas to draw on - you can reuse that when you modify what to draw below
c = ROOT.TCanvas("myCanvasName","The Canvas Title",800,600)
# Some styling options
ROOT.gStyle.SetOptStat(0)
ROOT.gStyle.SetOptTitle(False)

In [7]:
# Here we define what you want to look at:

# Add columns that contain the price in $-cent,
# keeping only those messages that belong to the ZCN5 contract.
rdfWithPrice = rdf.Filter('ZCN5_IsOriginalMessage == true') \
  .Define('AskCent', 'ZCN5_AskLevelPrices.AskLevelPrices[0] * ZCN5_TickSize') \
  .Define('BidCent', 'ZCN5_BidLevelPrices.BidLevelPrices[0] * ZCN5_TickSize')

# Separate bid- and ask-side messages.
rdfBidSideAction = rdfWithPrice.Filter('ActionSide == 0')
rdfAskSideAction = rdfWithPrice.Filter('ActionSide == 1')

# Create two histograms (40 bins, from $390 to $410) showing the ask price,
# one histogram for bid-side messages and one for ask-side messages
histBid = rdfBidSideAction.Histo1D(('hAskPrizeBid','bid side;ask price', 40, 390, 410), 'AskCent')
histAsk = rdfAskSideAction.Histo1D(('hAskPrizeAsk','ask side;ask price', 40, 390, 410), 'AskCent')

In [8]:
%jsroot on

# The above only defines what to do, it doesn't look at data yet.
# That happens only when drawing the histograms, here.
# So whenever you modify above what to draw you need to also re-run this
# cell, to actually do the analysis andd draw the result.
histBid.Draw()
histAsk.SetLineColor(ROOT.kRed)
histAsk.Draw("SAME")

# Build a little box that contains the histograms' titles.
legend = c.BuildLegend(0.7, 0.75, 0.85, 0.85)
legend.Draw()

# Draw the TCanvas on the Jupyter notebook. 
c.Draw()

In [9]:
# See which other columns exist:
file.LOB.Print()

******************************************************************************
*Tree    :LOB       : LOB                                                    *
*Entries :  5753585 : Total =     16984728009 bytes  File  Size =  255789446 *
*        :          : Tree compression factor =  66.51                       *
******************************************************************************
*Br    0 :Time      : Time/L                                                 *
*Entries :  5753585 : Total  Size=   46043029 bytes  File Size  =    2535935 *
*Baskets :      156 : Basket Size=    2697216 bytes  Compression=  18.15     *
*............................................................................*
*Br    1 :TradingSession : TradingSession/B                                  *
*Entries :  5753585 : Total  Size=    5757620 bytes  File Size  =      29290 *
*Baskets :       37 : Basket Size=     337408 bytes  Compression= 196.54     *
*...................................................